In [1]:
!apt-get update

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-10 16:47:59--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.42MB/s    in 0.6s    

2020-08-10 16:48:00 (1.42 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BDETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_US_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

mlrev_raw_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_multilingual_US_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
mlrev_raw_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   53096384| R63J84G1LOX6R|1563890119|     763187671|The Sandman Vol. ...|           Books|          4|            0|          1|   N|                N|ignore the review...|this is the first...|1995-08-13 00:00:00|
|         US|   53096399|R1BALOA11Z06MT|1559947608|     381720534|The 22 Immutable ...| 

In [5]:
mlrev_raw_df.count()

6931166

In [6]:
mlreviews = mlrev_raw_df.select(["customer_id", "review_id", "product_id", "product_category", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_body"])
mlreviews.show(5)

+-----------+--------------+----------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+
|customer_id|     review_id|product_id|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|         review_body|
+-----------+--------------+----------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+
|   53096384| R63J84G1LOX6R|1563890119|           Books|          4|            0|          1|   N|                N|this is the first...|
|   53096399|R1BALOA11Z06MT|1559947608|           Books|          4|            0|          0|   N|                N|I've always been ...|
|   53096332|R1LLAY5W5PZUS4|0671701800|           Books|          5|            1|          2|   N|                N|This is a book ab...|
|   53096335|R3R9VTJ82FXECQ|0425132153|           Books|          5|            0|          0|   N|                N|This is quite pos...|
|   51747709|R1P5J3FNBWTFXY

In [7]:
mlreviews.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_body: string (nullable = true)



In [8]:
mlrev = mlreviews.dropDuplicates(['customer_id', 'product_id'])
mlrev.show(5)

+-----------+--------------+----------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+
|customer_id|     review_id|product_id|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|         review_body|
+-----------+--------------+----------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+
|      10902|R3PMWI3X7E3YE9|0345803507|               Books|          5|            0|          1|   N|                Y|I Really loved th...|
|      11195|R1G3SGH52CMDNK|B00355L2TU|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I recommend this ...|
|      19112| R7K1AWEQCATEI|B000SW4DV2|Digital_Video_Dow...|          4|            0|          2|   N|                N|I give it a 4/5 i...|
|      21572|R1IX4INXCWS29Y|B008IJT0CE|         Mobile_Apps|          5|            0|          0|   N|                Y|I really like thi...|

In [10]:
mlrev.count()

6916537

In [12]:
mlrev_clean = mlrev[mlrev.star_rating.isNotNull()]


In [13]:
mlrev_clean.count()

6916536

In [14]:
mlrev_fin = mlrev_clean[mlrev_clean.verified_purchase.isNotNull()]
mlrev_fin.count()

6916536

In [19]:
# removed user and password for submission
mode = "append"
dburl = "jdbc:postgresql://<ENDPOINT>:5432/bigdatachallenge"
config = {"user":"",
          "password":"",
          "driver":"org.postgresql.Driver"}

In [20]:
mlrev_fin.write.jdbc(url=dburl, table='multilingual_reviews', mode=mode, properties=config)